# Model Performance Comparison & Visualizations

This notebook creates visualizations for comparing model performances, suitable for PowerPoint presentation.

**Models included:**
- Linear Regression (baseline)
- Random Forest
- XGBoost
- CatBoost
- LightGBM


In [ ]:
# === 1. Imports ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set style for professional presentations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Increase figure size and font for presentations
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12

print("✅ Imports complete")


In [ ]:
# === 2. Load Model Performance Data ===
# RMSE scores from validation/test sets (Kaggle test set)
model_performance = {
    'Linear Regression': 4103.64,  # Baseline from notebook 02
    'Random Forest': 2907,  # Kaggle test set RMSE (worst model)
    'XGBoost': 2476,  # From config
    'CatBoost': 1982,  # From config
    'LightGBM': 1892,  # From config (best individual model)
}

# Create DataFrame for easier manipulation
df_performance = pd.DataFrame({
    'Model': list(model_performance.keys()),
    'RMSE': list(model_performance.values())
})

# Sort by RMSE (best to worst)
df_performance = df_performance.sort_values('RMSE')

print("📊 Model Performance Data:")
print(df_performance.to_string(index=False))
print(f"\n✅ Best Model: {df_performance.iloc[0]['Model']} with RMSE = ${df_performance.iloc[0]['RMSE']:.2f}")
print(f"📉 Improvement over baseline: {((model_performance['Linear Regression'] - df_performance.iloc[0]['RMSE']) / model_performance['Linear Regression'] * 100):.1f}%")


## Visualization 1: RMSE Comparison Bar Chart


In [ ]:
# === 3. RMSE Comparison Bar Chart ===
fig, ax = plt.subplots(figsize=(14, 8))

# Create color gradient (green = best, red = worst)
colors = ['#2ecc71', '#3498db', '#9b59b6', '#e67e22', '#e74c3c']
bars = ax.barh(df_performance['Model'], df_performance['RMSE'], color=colors)

# Add value labels on bars
for i, (model, rmse) in enumerate(zip(df_performance['Model'], df_performance['RMSE'])):
    ax.text(rmse + 50, i, f'${rmse:.0f}', 
            va='center', fontsize=12, fontweight='bold')

# Customize chart
ax.set_xlabel('RMSE (Root Mean Squared Error)', fontsize=14, fontweight='bold')
ax.set_ylabel('Model', fontsize=14, fontweight='bold')
ax.set_title('Model Performance Comparison: RMSE Scores', 
             fontsize=18, fontweight='bold', pad=20)
ax.grid(axis='x', alpha=0.3, linestyle='--')

# Highlight best model
best_idx = df_performance['RMSE'].idxmin()
bars[df_performance.index.get_loc(best_idx)].set_edgecolor('gold')
bars[df_performance.index.get_loc(best_idx)].set_linewidth(3)

plt.tight_layout()
plt.savefig('../results/model_rmse_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: model_rmse_comparison.png")


## Visualization 2: Performance Improvement Over Baseline


In [ ]:
# === 4. Improvement Over Baseline ===
baseline_rmse = model_performance['Linear Regression']
df_performance['Improvement_%'] = ((baseline_rmse - df_performance['RMSE']) / baseline_rmse) * 100
df_performance['Improvement_$'] = baseline_rmse - df_performance['RMSE']

fig, ax = plt.subplots(figsize=(14, 8))

# Create horizontal bar chart
bars = ax.barh(df_performance['Model'], df_performance['Improvement_%'], 
               color=colors)

# Add value labels
for i, (model, pct, dollar) in enumerate(zip(df_performance['Model'], 
                                               df_performance['Improvement_%'],
                                               df_performance['Improvement_$'])):
    ax.text(pct + 0.5, i, f'{pct:.1f}% (${dollar:.0f})', 
            va='center', fontsize=12, fontweight='bold')

ax.set_xlabel('Improvement Over Baseline (%)', fontsize=14, fontweight='bold')
ax.set_ylabel('Model', fontsize=14, fontweight='bold')
ax.set_title('Model Performance Improvement Over Linear Regression Baseline', 
             fontsize=18, fontweight='bold', pad=20)
ax.grid(axis='x', alpha=0.3, linestyle='--')

plt.tight_layout()
plt.savefig('../results/model_improvement.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: model_improvement.png")


## Visualization 3: Prediction Distribution Comparison


In [ ]:
# === 5. Load Test Predictions for Distribution Analysis ===
results_dir = Path('../results')

# Load predictions from different models
predictions = {}

# LightGBM
lgbm_preds = pd.read_csv(results_dir / 'lightgbm_test_preds.csv')
predictions['LightGBM'] = lgbm_preds['Actual'].values

# CatBoost
catboost_preds = pd.read_csv(results_dir / 'catboost_test_preds.csv')
predictions['CatBoost'] = catboost_preds['Actual'].values

# XGBoost
xgb_preds = pd.read_csv(results_dir / 'xgb_test_preds.csv')
predictions['XGBoost'] = xgb_preds['Actual'].values

# Random Forest
rf_preds = pd.read_csv(results_dir / 'tuned_rf_submission.csv')
predictions['Random Forest'] = rf_preds['Actual'].values

print(f"✅ Loaded predictions for {len(predictions)} models")
print(f"   Sample size: {len(predictions['LightGBM'])} predictions per model")


In [ ]:
# === 6. Prediction Distribution Comparison ===
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

# Plot 1: Histogram overlay
ax1 = axes[0]
for model_name, preds in predictions.items():
    ax1.hist(preds, bins=50, alpha=0.6, label=model_name, density=True)
ax1.set_xlabel('Predicted Price ($)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Density', fontsize=12, fontweight='bold')
ax1.set_title('Prediction Distribution Comparison', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3)

# Plot 2: Box plot
ax2 = axes[1]
df_preds = pd.DataFrame(predictions)
box = ax2.boxplot([df_preds[col] for col in df_preds.columns], 
                  labels=df_preds.columns, patch_artist=True)
for patch, color in zip(box['boxes'], colors[:len(box['boxes'])]):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
ax2.set_ylabel('Predicted Price ($)', fontsize=12, fontweight='bold')
ax2.set_title('Prediction Distribution: Box Plot', fontsize=14, fontweight='bold')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(axis='y', alpha=0.3)

# Plot 3: Violin plot
ax3 = axes[2]
parts = ax3.violinplot([df_preds[col] for col in df_preds.columns], 
                       positions=range(len(df_preds.columns)),
                       showmeans=True, showmedians=True)
ax3.set_xticks(range(len(df_preds.columns)))
ax3.set_xticklabels(df_preds.columns, rotation=45, ha='right')
ax3.set_ylabel('Predicted Price ($)', fontsize=12, fontweight='bold')
ax3.set_title('Prediction Distribution: Violin Plot', fontsize=14, fontweight='bold')
ax3.grid(axis='y', alpha=0.3)

# Plot 4: Summary statistics
ax4 = axes[3]
ax4.axis('off')
stats_data = []
for model_name, preds in predictions.items():
    stats_data.append({
        'Model': model_name,
        'Mean': f"${preds.mean():,.0f}",
        'Median': f"${np.median(preds):,.0f}",
        'Std': f"${preds.std():,.0f}",
        'Min': f"${preds.min():,.0f}",
        'Max': f"${preds.max():,.0f}"
    })
stats_df = pd.DataFrame(stats_data)
table = ax4.table(cellText=stats_df.values, colLabels=stats_df.columns,
                  cellLoc='center', loc='center', bbox=[0, 0, 1, 1])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)
ax4.set_title('Prediction Summary Statistics', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig('../results/prediction_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: prediction_distributions.png")


## Visualization 4: Model Correlation Heatmap


In [ ]:
# === 7. Model Prediction Correlation ===
correlation_matrix = df_preds.corr()

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0.95, vmin=0.9, vmax=1.0, square=True, 
            linewidths=2, cbar_kws={'label': 'Correlation Coefficient'},
            ax=ax)
ax.set_title('Model Prediction Correlation Matrix', 
             fontsize=16, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig('../results/model_correlation.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: model_correlation.png")
print("\n📊 Correlation Insights:")
print(f"   High correlation ({correlation_matrix.min().min():.3f} - {correlation_matrix.max().max():.3f})")
print("   indicates models make similar predictions, limiting ensemble benefits.")


## Visualization 5: RMSE vs Model Complexity


In [ ]:
# === 8. Model Complexity vs Performance ===
# Define complexity scores (subjective, based on model type)
complexity_scores = {
    'Linear Regression': 1,  # Simplest
    'Random Forest': 3,
    'XGBoost': 4,
    'CatBoost': 4,
    'LightGBM': 4,
}

df_complexity = df_performance.copy()
df_complexity['Complexity'] = df_complexity['Model'].map(complexity_scores)

fig, ax = plt.subplots(figsize=(12, 8))

# Scatter plot
scatter = ax.scatter(df_complexity['Complexity'], df_complexity['RMSE'], 
                    s=300, c=colors, alpha=0.7, edgecolors='black', linewidth=2)

# Add labels
for idx, row in df_complexity.iterrows():
    ax.annotate(row['Model'], 
               (row['Complexity'], row['RMSE']),
               xytext=(5, 5), textcoords='offset points',
               fontsize=11, fontweight='bold')

ax.set_xlabel('Model Complexity (1=Simple, 4=Complex)', fontsize=14, fontweight='bold')
ax.set_ylabel('RMSE ($)', fontsize=14, fontweight='bold')
ax.set_title('Model Performance vs Complexity Trade-off', 
             fontsize=16, fontweight='bold', pad=20)
ax.grid(alpha=0.3, linestyle='--')
ax.set_xticks(range(1, 5))

plt.tight_layout()
plt.savefig('../results/complexity_vs_performance.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: complexity_vs_performance.png")


## Visualization 6: Performance Ranking


In [ ]:
# === 9. Performance Ranking Chart ===
fig, ax = plt.subplots(figsize=(12, 8))

# Create ranking (1 = best)
df_performance['Rank'] = range(1, len(df_performance) + 1)

# Horizontal bar chart
bars = ax.barh(df_performance['Model'], df_performance['RMSE'], 
               color=colors, alpha=0.8)

# Add rank labels
for i, (model, rmse, rank) in enumerate(zip(df_performance['Model'], 
                                             df_performance['RMSE'],
                                             df_performance['Rank'])):
    # Rank badge
    ax.text(rmse - 200, i, f'#{rank}', 
            va='center', ha='right', fontsize=16, fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='white', edgecolor='black', pad=0.5))
    # RMSE value
    ax.text(rmse + 50, i, f'${rmse:.0f}', 
            va='center', fontsize=12, fontweight='bold')

ax.set_xlabel('RMSE ($)', fontsize=14, fontweight='bold')
ax.set_ylabel('Model', fontsize=14, fontweight='bold')
ax.set_title('Model Performance Ranking', fontsize=18, fontweight='bold', pad=20)
ax.grid(axis='x', alpha=0.3, linestyle='--')

# Invert y-axis so best is at top
ax.invert_yaxis()

plt.tight_layout()
plt.savefig('../results/model_ranking.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: model_ranking.png")


## Summary Statistics


In [ ]:
# === 10. Final Summary Table ===
print("="*70)
print("MODEL PERFORMANCE SUMMARY")
print("="*70)
print(f"\n{'Model':<25} {'RMSE ($)':<15} {'Rank':<10} {'Improvement %':<15}")
print("-"*70)

for idx, row in df_performance.iterrows():
    improvement = ((baseline_rmse - row['RMSE']) / baseline_rmse) * 100
    print(f"{row['Model']:<25} ${row['RMSE']:<14.0f} {row['Rank']:<10} {improvement:<14.1f}%")

print("\n" + "="*70)
print(f"🏆 Best Model: {df_performance.iloc[0]['Model']}")
print(f"   RMSE: ${df_performance.iloc[0]['RMSE']:.2f}")
print(f"   Improvement over baseline: {((baseline_rmse - df_performance.iloc[0]['RMSE']) / baseline_rmse * 100):.1f}%")
print(f"   Absolute improvement: ${baseline_rmse - df_performance.iloc[0]['RMSE']:.2f}")
print("="*70)

# Save summary to CSV
df_performance.to_csv('../results/model_performance_summary.csv', index=False)
print("\n✅ Summary saved to: model_performance_summary.csv")


## Generated Files

All visualizations have been saved to the `results/` folder:

1. **model_rmse_comparison.png** - Bar chart comparing RMSE scores
2. **model_improvement.png** - Improvement over baseline
3. **prediction_distributions.png** - Distribution comparisons
4. **model_correlation.png** - Correlation heatmap
5. **complexity_vs_performance.png** - Complexity trade-off
6. **model_ranking.png** - Performance ranking
7. **model_performance_summary.csv** - Summary statistics

These files are ready for use in your PowerPoint presentation!
